# Introduction and setup

In this notebook, I have code to load in the stellar profiles in 3 dimensions which 
were previously calculated on the command line so that we can plot quantities such as 
the 1D velocity dispersion and evolution with time of the mass.

In [ ]:
using Revise
using LilGuys
using CairoMakie, Arya

In [ ]:
modelsdir = "/astro/dboyea/dwarfs"

In [ ]:
function load_profiles(filename, parampath=nothing; extension="stellar_profiles_3d.hdf5")
    path = joinpath(modelsdir, filename, extension)

    if parampath === nothing
        parampath = joinpath(modelsdir, filename * "/profile.toml")
    end
        
    if isfile(parampath)
        expected = LilGuys.load_profile(parampath)
    else
        expected = nothing
    end

    profs = LilGuys.read_structs_from_hdf5(path, LilGuys.StellarProfile3D)
    idxs = parse.(Int, first.(profs))
    profs = last.(profs)

    s = sortperm(idxs)
    return [idxs[i] => profs[i] for i in s ], expected
end

In [ ]:
function compare_profiles(profiles, expected=nothing; 
        sequential=false, legend=true, limits=(-1.5, 0.8, -15, 3), 
        kwargs...
    )
    
    fig = Figure()

	ax = Axis(fig[1,1], xlabel=L"\log\, r / \textrm{kpc}", ylabel =  L"\log\, \rho_\star\; [10^{10} M_\odot / \textrm{kpc}^3]", 
		limits=limits; kwargs...
		)

    plot_kwargs = Dict{Symbol, Any}()

    if sequential
        plot_kwargs[:colorrange] = (1, length(profiles))
    end
    
    for i in eachindex(profiles)
        label, prof = profiles[i]
        x = prof.log_r
        y = log10.(prof.rho)
        if sequential
            plot_kwargs[:color] = i
        end

    	lines!(x, y, label=string(label); plot_kwargs...)
    end

    if expected !== nothing
    	log_r_pred = LinRange(-2, 2, 1000)
    	ρ_s_pred = calc_ρ.(expected, 10 .^ log_r_pred)

    	lines!(log_r_pred, log10.(ρ_s_pred), label="expected", color="black", linestyle=:dot)
    end

    if legend
    	axislegend(ax, position=:lb)
    end 
    
	fig
end

In [ ]:
function plot_v_t!(profiles; kwargs...)
	sigmas = [p.second.sigma_vx for p in profiles]

    t = [p.second.time for p in profiles] 
	scatterlines!(t * T2GYR, sigmas * V2KMS; kwargs...)
end

In [ ]:
function plot_v_t(profiles; kwargs...)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}";
        kwargs...
	)

    plot_v_t!(profiles)

    fig
end
    

In [ ]:
function compare_v_t(profilesi)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}",
	)


    for i in eachindex(profilesi)
        label, profiles = profilesi[i]

        kwargs = Dict{Symbol, Any}()
        kwargs[:label] = label
  
        plot_v_t!(profiles; kwargs...)
    end

    axislegend()
	fig
end

In [ ]:
function plot_r_percens(profiles; kwargs...)
	fig = Figure()
	ax = Axis(fig[1,1], xlabel="time / Gyr", ylabel="log r / kpc containing stellar mass"; kwargs...)

	q = profiles[1].second.quantiles
	t = [p.second.time for p in profiles] * T2GYR
    
	for i in reverse(eachindex(q))
		Mq = [log10.(p.second.r_quantile[i]) for p in profiles]
        
		lines!(t, Mq, color=q[i], colorrange=extrema(q), label="$(q[i])")
	end

	Legend(fig[1, 2], ax, "quantile")
	fig
end
    

In [ ]:
"""
    all_plots(profiles[, expected])

Given the profiles object, this function displayes the plots
for the profiles and r percentiles for each timestep in the profile

"""
function all_plots(profiles, expected=nothing; prof_limits=(-2, 1, -15, 3), title="")
   display(compare_profiles(profiles, expected, 
            sequential=true, legend=false, title=title, limits=prof_limits,
        )
    )
    
    display(plot_r_percens(profiles, title=title))

    display(plot_v_t(profiles, title=title))

end

# 1e6 fiducial halo

In [ ]:
model = "analysis/sculptor/1e6_V31_r5.9/stars/"

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.13"));

In [ ]:
all_plots(profs, expected)

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.10"));

In [ ]:
all_plots(profs, expected)

In [ ]:
all_plots(profs, expected)

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.07"));

In [ ]:
all_plots(profs, expected)

## Heavy

In [ ]:
model = "analysis/sculptor/1e6_V40_r5.9/stars/"

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.10"));

In [ ]:
all_plots(profs, expected)

### Rs = 0.05

In [ ]:
ana_profs, _ = load_profiles(joinpath(model, "ana_stars/exp2d_rs0.10"));

In [ ]:
profs, expected = load_profiles(joinpath(model, "stars/exp2d_rs0.10"));

In [ ]:
profs_o, _ = load_profiles(joinpath(model, "stars_old/exp2d_rs0.10"));

In [ ]:
all_profs_end = [
    "emperical" => profs_o[end].second,
    "abs(emperical)" => pos_profs[end].second,
    "analytic" => ana_profs[end].second,
    ];

In [ ]:
compare_profiles(all_profs_end, expected, title="Rs=0.05")

In [ ]:
compare_v_t(all_profs, times_Gyr)


In [ ]:
plot_r_percens(old_profs, times_Gyr)

In [ ]:
plot_r_percens(profs, idxs)

In [ ]:
plot_r_percens(ana_profs, ana_idxs)

### Rs = 0.30

In [ ]:
ana_profs, _ = load_profiles(joinpath(model, "stars_ana/exp2d_rs0.20"));

In [ ]:
plot_r_percens(ana_profs)

## King

### Rs=0.30

In [ ]:
ana_profs, expected = load_profiles(joinpath(model, "stars_ana/king_rs0.30"));

In [ ]:
compare_profiles(ana_profs, expected, sequential=true)

In [ ]:
plot_r_percens(ana_profs)

### Rs=0.15

In [ ]:
ana_profs, _ = load_profiles(joinpath(model, "stars_ana/king_rs0.15"));

In [ ]:
pos_profs, expected = load_profiles(joinpath(model, "stars/king_rs0.15"));

In [ ]:
old_profs, _ = load_profiles(joinpath(model, "stars_old/king_rs0.15"));

In [ ]:
all_profs = [
    "emperical" => old_profs,
    "abs(emperical)" => pos_profs,
    "analytic" => ana_profs,
    ];

In [ ]:
all_profs_end = [
    "emperical" => old_profs[end].second,
    "abs(emperical)" => pos_profs[end].second,
    "analytic" => ana_profs[end].second,
    ];

In [ ]:
all_profs_begin = [
    "emperical" => old_profs[1].second,
    "abs(emperical)" => pos_profs[1].second,
    "analytic" => ana_profs[1].second,
    ];

In [ ]:
compare_profiles(all_profs_end, expected, title="King")

In [ ]:
compare_profiles(all_profs_begin, expected, title="King")

In [ ]:
compare_profiles(ana_profs, expected, sequential=true)

In [ ]:
compare_v_t(all_profs)

### Rs=0.10

In [ ]:
ana_profs, _ = load_profiles(joinpath(model, "stars_ana/king_rs0.10"));

In [ ]:
pos_profs, expected = load_profiles(joinpath(model, "stars/king_rs0.10"));

In [ ]:
old_profs, _ = load_profiles(joinpath(model, "stars_old/king_rs0.10"));

In [ ]:
all_profs = [
    "emperical" => old_profs,
    "abs(emperical)" => pos_profs,
    "analytic" => ana_profs,
    ];

In [ ]:
all_profs_end = [
    "emperical" => old_profs[end].second,
    "abs(emperical)" => pos_profs[end].second,
    "analytic" => ana_profs[end].second,
    ];

In [ ]:
all_profs_begin = [
    "emperical" => old_profs[3].second,
    "abs(emperical)" => pos_profs[3].second,
    "analytic" => ana_profs[3].second,
    ];

In [ ]:
compare_v_t([profs_o, profs, ana_profs], [idxs_o, idxs, ana_idxs], times_Gyr, ["emperical", "abs(emperical)", "analytic"])


In [ ]:
compare_profiles(ana_profs, expected, sequential=true)

In [ ]:
plot_r_percens(old_profs, times_Gyr)

In [ ]:
plot_r_percens(profs, idxs)

In [ ]:
plot_r_percens(ana_profs, ana_idxs)

## king individual profiles

In [ ]:
profs, expected = load_profiles(joinpath(model, "stars_ana/king_rs0.15"));

In [ ]:
all_plots(profs, expected)

## Alternate halos

In [ ]:
model = "analysis/sculptor/1e6_V31_r3.2/stars/"

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.13"));

In [ ]:
all_plots(profs, expected)

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.10"));

In [ ]:
all_plots(profs, expected)

In [ ]:
compact_profs, expected = load_profiles(joinpath(model, "exp2d_rs0.05"));

In [ ]:
all_plots(compact_profs, expected)

In [ ]:
profs, expected = load_profiles(joinpath(model, "king_rs0.10"));

In [ ]:
expected.R_s, expected.R_t

In [ ]:
all_plots(profs, expected)

In [ ]:
all_profs = [
    "fiducial" => profs,
    "compact"  => compact_profs
    ];

In [ ]:
compare_v_t(all_profs)

In [ ]:
all_profs_end = [
    "fiducial" => profs[end].second,
    "compact"  => compact_profs[end].second,
    ];

## 1e7

In [ ]:
model = "analysis/sculptor/1e7_V31_r5.9/stars/"

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.10"));
label = L"1e7, Exp2D($R_s=0.10$ kpc)"

In [ ]:
all_plots(profs, expected, title=label)

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.05"));

label = L"1e7, Exp2D($R_s=0.05$ kpc)"
expected

In [ ]:
all_plots(profs, expected, title=label)

In [ ]:
profs, expected = load_profiles(joinpath(model, "king_rs0.10"));
label = L"1e7, King($R_s=0.1,\ R_t=0.5$ kpc)"
expected.R_s,

In [ ]:
all_plots(profs, expected, title=label, prof_limits=(-2, 0, -5, 3))

# Orbit studies

In [ ]:
model = "analysis/sculptor/1e6_V31_r3.2/orbit1/stars/"

In [ ]:
profs, expected = load_profiles(joinpath(model, "exp2d_rs0.10"));

In [ ]:
all_plots(profs, expected)